# Solving delay differential equations

<hr>

In [1]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

import numpy as np

import biocircuits

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

The `biocircuits.dde()` function solves delay differential equations. The development of this function is described in [the lesson on delay oscillators](../lessons/09_delay_oscillators.ipynb#Numerical-solution-of-DDEs).

As an example 

In [98]:
# Right hand side of simple delay oscillator
def delay_rhs(xy, t, xy_past, beta, n, tau_x, tau_y):
    x, y = xy
    x_past, _ = xy_past(t - tau_x)
    _, y_past = xy_past(t - tau_y)
    
    return np.array(
        [
            beta / (1 + x_past ** n) - x,
#           beta * x_past ** n / (1 + x_past ** n + y_past ** n) - y,
            beta / (1 + y_past ** n) - y,
        ]
    )


# Initially, we just have no gene product
def xy0(t):
    return np.array([0.0, 0.0])


# Specify parameters (All dimensionless)
beta = 4.0
n = 2.0
tau_x = 10.0
tau_y = 25.0

# Time points we want
t = np.linspace(0, 200, 400)

# Perform the integration
xy = biocircuits.ddeint(
    delay_rhs, xy0, t, max(tau_x, tau_y), args=(beta, n, tau_x, tau_y)
)

# Plot the result
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=200,
    x_axis_label="time",
    y_axis_label="x",
    x_range=[t.min(), t.max()],
)
p.line(t, xy[:, 0], line_width=2)
p.line(t, xy[:, 1], line_width=2, line_color='orange')

bokeh.io.show(p)

25.0


In [82]:
# Right hand side of simple delay oscillator
def delay_rhs(x, t, x_past, beta, n, tau):
    return beta / (1 + x_past(t - tau) ** n) - x


# Initially, we just have no gene product
def x0(t):
    return 0


# Specify parameters (All dimensionless)
beta = 4
n = 2
tau = 10

# Time points we want
t = np.linspace(0, 200, 400)

# Perform the integration
x = biocircuits.ddeint(delay_rhs, x0, t, tau, args=(beta, n, tau))

# Plot the result
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=200,
    x_axis_label="time",
    y_axis_label="x",
    x_range=[t.min(), t.max()],
)
p.line(t, x[:, 0], line_width=2)

bokeh.io.show(p)

## Our model system

As we learn how to use some Python-based tools for analysis of genetic circuits, we will learn an important design principle about *cascades*. A three-component cascade is illustrated below.

\begin{align}
\mathrm{X} \longrightarrow \mathrm{Y} \longrightarrow \mathrm{Z}
\end{align}

Here, X is our input, which we will specify. X could be something like an externally imposed stimulus.  We are interested in the response of Y and Z as a function of input X. We will expose the design principle that *a cascade can filter out high frequency fluctuations*. This makes sense intuitively. Imagine we have a brief pulse in X. Y will start to grow in concentration during the pulse, but will then fall back toward its basal level when the pulse stops. If there is not enough time for Y to accumulate enough to enhance production of Z, then the level of Z does not change appreciably. Thus, Z shows no response to a brief pulse.

## The dynamical equations

We will model the circuit by writing down a system of ordinary differential equations and solving them. We will assume Hill-like behavior for the activation of Y by X and of Z by Y. We also neglect leakage. We define the concentrations of X, Y, and Z, respectively as $x$, $y$, and $z$. The system of ODEs describing this system is then

\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= \beta_y\,\frac{(x/k_x)^{n_x}}{1+(x/k_x)^{n_x}} - \gamma_y y, \\[1em]
\frac{\mathrm{d}z}{\mathrm{d}t} &= \beta_z\,\frac{(y/k_y)^{n_y}}{1+(y/k_y)^{n_y}} - \gamma_z z.
\end{align}

Note that the imposed $x(t)$ is in general a function of time.  

## Nondimensionalization

As is generally a good idea for analysis of these systems, we will **nondimensionalize** the dynamical equations. The variables and parameters in the problem have **dimension**. For example, $y$ has dimension of concentration, or number of particles per cubic length, written as

\begin{align}
y = \left[\frac{N}{L^3}\right].
\end{align}

The parameter $\gamma_y$ has dimension of inverse time, or $\gamma_y = [T^{-1}]$. It is clear that each term in the dynamical equations has dimension of $N/L^3 T$. In general, every term in an equation must have the same dimension; trying to add, for example, a meter to a kilogram is nonsensical.

The nondimensionalization procedure involves rewriting the equations such that every term is **dimensionless**, which means that every term has dimension of unity. This is beneficial for several reasons.

1. It reduces the number of parameters you need to consider. Only dimensionless ratios/products of parameters are considered.
2. It allows comparison of the relative magnitudes of terms in an equation.
3. It provides intuition by identifying which ratios and products of parameters determine the dynamics.

There is no one way to nondimensionalize a set of dynamical equations. We have found the following procedure to be generally effective. Let $\theta$ be one of the variables, space, or time. In the above dynamical equations, $\theta$ could be $x$, $y$, $z$, or $t$. (Here $x$, $y$, and $z$ denote concentrations; there are no spatial variables in this problem.) By contrast, $\beta_y$, $\beta_z$, $k_x$, $k_y$, $n_x$, $n_y$, $\gamma_y$, and $\gamma_z$ are system *parameters*, not variables.

1. For each variable and time, define a dimensionless version (usually marked by a tilde) such that $\theta = \theta_d \tilde{\theta}$, where $\theta$ is some variable or time. The constant $\theta_d$ imparts the dimension on $\theta$ (hence the subscript *d*).
2. Substitute these expressions into the dynamical equations.
3. Rearrange the equations such that every term is dimensionless. As a result, you find ratios and products of the $\theta_d$'s and the system parameters. These ratios are dimensionless and are called *dimensionless parameters*.
4. Choose expressions for the $\theta_d$'s that minimize the number of dimensionless parameters.

As usual, this is best seen by example. We start be defining

\begin{align}
&t = t_d\,\tilde{t},\\[1em]
&x = x_d\,\tilde{x},\\[1em]
&y = y_d\,\tilde{y},\\[1em]
&z = z_d\,\tilde{z}.
\end{align}

Inserting these into the dynamical equations gives

\begin{align}
&\frac{y_d}{t_d}\,\frac{\mathrm{d}\tilde{y}}{\mathrm{d}\tilde{t}} = \beta_y\,\frac{(x_d\tilde{x}/k_x)^{n_x}}{1+(x_d\tilde{x}/k_x)^{n_x}} - \gamma_y y_d \tilde{y}, \\[1em]
&\frac{z_d}{t_d}\frac{\mathrm{d}\tilde{z}}{\mathrm{d}\tilde{t}} = \beta_z\,\frac{(y_d\tilde{y}/k_y)^{n_y}}{1+(y_d\tilde{y}/k_y)^{n_y}} - \gamma_z z_d\tilde{z}.
\end{align}

To make every term dimensionless, we can divide the top equation by $y_d/t_d$ and the bottom equation by $z_d/t_d$. The result is

\begin{align}
&\frac{\mathrm{d}\tilde{y}}{\mathrm{d}\tilde{t}} = \frac{\beta_y\,t_d}{y_d}\,\frac{(x_d\tilde{x}/k_x)^{n_x}}{1+(x_d\tilde{x}/k_x)^{n_x}} - \gamma_y t_d \tilde{y}, \\[1em]
&\frac{\mathrm{d}\tilde{z}}{\mathrm{d}\tilde{t}} = \frac{\beta_z\,t_d}{z_d}\,\frac{(y_d\tilde{y}/k_y)^{n_y}}{1+(y_d\tilde{y}/k_y)^{n_y}} - \gamma_z t_d\tilde{z}.
\end{align}

We identify our dimensionless parameters as
\begin{align}
&\beta_y\,t_d/y_d, \\[1em]
&\beta_z\,t_d/z_d, \\[1em]
&x_d/k_x,\\[1em]
&y_d/k_y,\\[1em]
&\gamma_y t_d,\\[1em]
&\gamma_z t_d,
\end{align}

with $n_x$ and $n_y$ already being dimensionless. We see immediately that we can eliminate two of the dimensionless parameters if we choose $x_d = k_x$ and $y_d = k_y$. We can eliminate another by choosing $t_d = 1/\gamma_y$. (We could have also chosen $t_d = 1 / \gamma_z$.) Thus, we have $\tilde{\gamma} \equiv \gamma_z / \gamma_y$ and $\tilde{\beta} \equiv \beta_y/\gamma_y k_y$ as a dimensionless parameters. Finally, we can choose $z_d = \beta_z/\gamma_z$ such that $\beta_z\,t_d/z_d = \tilde{\gamma}$. This last choice does not add a dimensionless parameter, since we have already defined $\tilde{\gamma}$.

Thus, our dimensionless equations are

\begin{align}
&\frac{\mathrm{d}\tilde{y}}{\mathrm{d}\tilde{t}} = \tilde{\beta}\,\frac{\tilde{x}^{n_x}}{1+\tilde{x}^{n_x}} - \tilde{y}, \\[1em]
&\frac{\mathrm{d}\tilde{z}}{\mathrm{d}\tilde{t}} = \tilde{\gamma}\,\frac{\tilde{y}^{n_y}}{1+\tilde{y}^{n_y}} - \tilde{\gamma} \tilde{z}.
\end{align}

For notational convenience, and since we will always be working in dimensionless units, we will drop all of the tildes. Doing this and dividing the bottom equation by $\tilde{\gamma}$ gives

\begin{align}
\frac{\mathrm{d}y}{\mathrm{d}t} &= \beta \,\frac{x^{n_x}}{1+x^{n_x}} - y, \\[1em]
\gamma^{-1}\,\frac{\mathrm{d}z}{\mathrm{d}t} &= \frac{y^{n_y}}{1+y^{n_y}} - z.
\end{align}

Thus, in addition to the specifics of our input $x(t)$, we have four parameters, $\beta$, $\gamma$, $n_x$, and $n_y$. We already know that $n_x$ and $n_y$ are Hill coefficients that parametrize the ultrasensitivity of the regulation by X and Y respectively. The parameter $\beta = \beta_y/\gamma_y k_y$ is the ratio of the unregulated level of Y, $\beta_y/\gamma$ to the level of Y that activates Z. The parameter $\gamma$ is the ratio of the decay rate of Z to that of Y. If dilution is the primary mode of concentration depletion for both Y and Z, then $\gamma \approx 1$. We now have a clear picture of which dimensionless parameters dictate the dynamics and what they mean physically.

## The scipy.intergrate module

Now that we have our dimensionless dynamical equations, we will move to solving them.

**The [SciPy Library](https://docs.scipy.org/doc/scipy/reference/)** is a Python library for scientific computing. It contains many modules, including `scipy.stats`, `scipy.special`, and `scipy.optimize`, which respectively have functions to perform statistical calculations, special functions, and optimization routines. There are many more. We will use the `scipy.integrate` module to integrate systems of ODEs.

There are three main APIs for solving real-valued initial value problems in the module. They are [solve_ivp()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp), [ode()](https://scipy.github.io/devdocs/generated/scipy.integrate.ode.html#scipy.integrate.ode), and [odeint()](https://scipy.github.io/devdocs/generated/scipy.integrate.odeint.html#scipy.integrate.odeint). According to the SciPy developers, `solve_ivp()` is the preferred method, with the others labeled as having an "old" API. The `solve_ivp()` function has the flexibility of allowing choice of multiple numerical algorithms for solving ODEs. However, for the kinds of problems we encounter in this class, I find that the generic LSODA algorithm developed by Linda Petzold and Alan Hindmarsh that handles both stiff and non-stiff problems with variable time stepping is the best option. This is the only solver offered in the `odeint()` function. If we compare the two solvers, `solve_ivp()` and `odeint()`, the former has a large overhead, which can lead to performance issues for small problems (for large problems, this is not a big deal). Since most of our problems are small, we will use `odeint()`. It has much better performance, and though its API is different, it is still intuitive.

The basic call signature for `odeint()` is

    scipy.integrate.odeint(func, y0, t, args=())

There are many other keyword arguments to set algorithmic parameters, but we will generally not need them (and you can read about them in the [documentation](https://scipy.github.io/devdocs/generated/scipy.integrate.odeint.html#scipy.integrate.odeint)).
Importantly, `func` is a vector-valued function with call signature `func(y, t, *args)` that specifies the right hand side of the system of ODEs to be solved. `t` is a scalar time point and `y` is a one-dimensional array (though multidimensional arrays are possible). `y0` is an array with the initial conditions.

As is often the case, use of this function is best seen by example, and we will now apply it to the cascade circuit.

## Solving for a constant input X

We will first consider the case where we initially have no X, Y, or Z present. At time $t = 0$, we suddenly have a concentration of X of $x_0$. So, we need five parameters for the right hand side of our ODEs, $\beta$, $\gamma$, $n_x$, $n_y$, and $x_0$.

We now define the function for the right hand side of the ODEs.

In [2]:
def cascade_rhs(yz, t, beta, gamma, n_x, n_y, x):
    """
    Right hand side for cascade X -> Y -> Z.  Return dy/dt and dz/dt.
    """
    # Unpack y and z
    y, z = yz

    # Compute dy/dt
    dy_dt = beta * x ** n_x / (1 + x ** n_x) - y

    # Compute dz/dt
    dz_dt = gamma * (y ** n_y / (1 + y ** n_y) - z)

    # Return the result as a NumPy array
    return np.array([dy_dt, dz_dt])

We can now define the initial conditions, our parameters, and the time points we want and use `scipy.integrate.odeint()` to solve.

In [3]:
# Number of time points we want for the solutions
n = 400

# Time points we want for the solution
t = np.linspace(0, 10, n)

# Initial condition
yz_0 = np.array([0.0, 0.0])

# Parameters
beta = 1.0
gamma = 1.0
n_x = 2
n_y = 2
x0 = 2.0

# Package parameters into a tuple
args = (beta, gamma, n_x, n_y, x0)

# Integrate ODES
yz = scipy.integrate.odeint(cascade_rhs, yz_0, t, args=args)

That's it!  The integration is done.  We can now look at what `scipy.integrate.odeint()`'s output looks like.

In [4]:
yz.shape

(400, 2)

The first column of the output `yz` gives $y(t)$ at the specified time points and the second column gives $z(t)$. We would now like to plot the results.

## Plotting results

We will use [Bokeh](https://docs.bokeh.org/) to plot the results. The syntax is pretty self-explanatory from the example below. Note that you can save a plot as a PNG by clicking the disk icon next to the plot, which might be helpful for incorporating your plots into your presentations. (Note that for publications, you should usually save your figures in a vector graphics format, which [Bokeh supports](https://docs.bokeh.org/en/latest/docs/user_guide/export.html), but is not necessary, and in fact discouraged, for display of plots in the browser.) 

Before building the plot, we will load in the color scheme we will use. The [colorcet package](https://colorcet.holoviz.org/) is good for this. I like to use the Category10 color palette for categorical colors.

In [5]:
# Set up color palette for this notebook
colors = colorcet.b_glasbey_category10

Colors in place, we will write a function to set up the plot. Within this function, we will write the solver; not just the plotting function. It will become clear that this is useful for interactive graphics. Because transcription rates, decay rates, and input concentrations of X can vary over orders of magnitude, we will input those as their logarithms. Finally, we will allow for normalization of the responses. That is, we allow for rescaling of the Y and Z concentrations such that their maximum concentration is one.

In [6]:
def cascade_response_plot(
    log_beta=0,
    log_gamma=0,
    n_x=2,
    n_y=2,
    log_x0=np.log10(2),
    t_max=10,
    normalize=False,
):
    """Generate a plot showing response of the cascade circuit
    to a step in x0.
    """
    # Package parameters into a tuple
    args = (
        10 ** log_beta,
        10 ** log_gamma,
        n_x,
        n_y,
        10 ** log_x0,
    )

    # Integrate ODES
    t = np.linspace(0, t_max, 400)
    yz = scipy.integrate.odeint(cascade_rhs, yz_0, t, args=args)
    y, z = yz.transpose()

    # Normalize Y and Z responses
    if normalize:
        y /= y.max()
        z /= z.max()

    # Set up plot
    p = bokeh.plotting.figure(
        frame_width=325,
        frame_height=250,
        x_axis_label="dimensionless time",
        y_axis_label=f"{'normalized ' if normalize else ''}dimensionless y, z",
        x_range=[0, t_max],
    )

    # Populate glyphs
    p.line(t, y, line_width=2, color=colors[0], legend_label="y")
    p.line(t, z, line_width=2, color=colors[1], legend_label="z")

    # Place the legend
    p.legend.location = "bottom_right"

    return p

Now that we have our function, we call it to generate the graphic, and then we use `bokeh.io.show()` to display the graphic in the notebook. Note that at the top of the notebook, we called `bokeh.io.output_notebook()` which tells `bokeh.io.show()` to display the plot in the notebook instead of writing it out to a file.

In [7]:
p = cascade_response_plot()

# Show plot
bokeh.io.show(p)

We see that the cascade acts as a delay for changes in $z$ as a result of input $x$.

## Interactive plotting and varying parameters

Plotting with Bokeh in Jupyter notebooks allows for interactivity with plots that can help to rapidly gain insights about how parameter values might affect the dynamics. We have found that this is a useful tool to rapidly explore parameter dependence on circuit behavior. We will construct an interactive plot for the cascade circuit to demonstrate how it is done.

For our interactivity, we will use [Panel](https://panel.holoviz.org). (Note that we imported Panel as `pn` at the beginning of this notebook and executed `pn.extension()` to enable Panel to work.) Panel allows convenient functionality for making interactive graphics. As you will see throughout the course, interactive graphics make it easier to explore how circuits work; you can interact directly with plots instead of having to re-run calculations and manually re-plot.

<div class="alert alert-info">

**Note:** As of Panel version 0.10.3, Panel graphics do not natively render in JupyterLab version 3.0.0 and above. We expect this to be fixed in short order, so we proceed with code cells as we would once these kinks are ironed out in this and future lessons. 
   
Note that because of this, the plots will not render in the rendered HTML version of this notebook. The plots would not be responsive anyway because there is no Python engine in the static HTML rendering. Nonetheless, as described below, you can still interact with your plots by serving them and using the `servable()` method of Panel objects.
    
</div>

As a first step in creating a graphic, we create **sliders**, which enable us to graphically change values of parameters. I'll first build an example slider in Panel.

In [8]:
pn.widgets.FloatSlider(name='example', start=0, end=1, step=0.01, value=0.5)

FloatSlider(name='example', step=0.01, value=0.5, value_throttled=0.5)

We would like to build such sliders for all of the parameter of our model.

In [9]:
log_beta_slider = pn.widgets.FloatSlider(
    name="log₁₀ β", start=-1, end=2, step=0.1, value=0
)
log_gamma_slider = pn.widgets.FloatSlider(
    name="log₁₀ γ", start=-1, end=2, step=0.1, value=0
)
nx_slider = pn.widgets.FloatSlider(
    name="nx", start=0.1, end=10, step=0.1, value=2
)
ny_slider = pn.widgets.FloatSlider(
    name="ny", start=0.1, end=10, step=0.1, value=2
)
log_x0_slider = pn.widgets.FloatSlider(
    name="log₁₀ x0", start=-1, end=2, step=0.1, value=np.log10(2)
)
t_max_slider = pn.widgets.FloatSlider(
    start=1, end=20, step=0.1, value=10, name="t_max",
)

The `normalize` parameter is a binary choice, so we cannot use a slider for it. Instead, we can use a checkbox.

In [10]:
normalize_checkbox = pn.widgets.Checkbox(name='Normalize', value=False)

Once the sliders and checkbox are built, adding the interactivity to the plot is as simple as adding a `@pn.depends()` decorator to the plotting function. Specifically, we set things up so that the parameters of the function that generates the plot depend on the values of the sliders. We simply add the decorator to the function we already wrote to do this.

In [11]:
@pn.depends(
    log_beta_slider.param.value,
    log_gamma_slider.param.value,
    nx_slider.param.value,
    ny_slider.param.value,
    log_x0_slider.param.value,
    t_max_slider.param.value,
    normalize_checkbox.param.value,
)
def cascade_response_plot(
    log_beta=0,
    log_gamma=0,
    n_x=2,
    n_y=2,
    log_x0=np.log10(2),
    t_max=10,
    normalize=False,
):
    # Package parameters into a tuple
    args = (
        10 ** log_beta,
        10 ** log_gamma,
        n_x,
        n_y,
        10 ** log_x0,
    )

    # Integrate ODES
    t = np.linspace(0, t_max, 400)
    yz = scipy.integrate.odeint(cascade_rhs, yz_0, t, args=args)
    y, z = yz.transpose()

    # Normalize Y and Z responses
    if normalize:
        y /= y.max()
        z /= z.max()

    # Set up plot
    p = bokeh.plotting.figure(
        frame_width=325,
        frame_height=250,
        x_axis_label="dimensionless time",
        y_axis_label=f"{'normalized ' if normalize else ''}dimensionless y, z",
        x_range=[0, t_max],
    )

    # Populate glyphs
    p.line(t, y, line_width=2, color=colors[0], legend_label="y")
    p.line(t, z, line_width=2, color=colors[1], legend_label="z")

    # Place the legend
    p.legend.location = "bottom_right"

    return p

Now, we need to lay things out. We will construct the widgets (the sliders) in a column using the `pn.Column()` class and then build the plot next to the widgets using the `pn.Row()` class. I'll add a little spacer to keep things well-spaced.

<!--
**Note: The interactivity from Panel will not be available in the HTML version of this notebook because Python needs to be running to perform the updates. All subsequent interactive plots will appear static in the HTML rendering of this notebook.**
-->

In [12]:
widgets = pn.Column(
    pn.Spacer(height=10),
    log_beta_slider,
    log_gamma_slider,    
    nx_slider,
    ny_slider,
    t_max_slider,
    log_x0_slider,
    width=150,
)

left_column = pn.Column(
    pn.Row(pn.Spacer(width=30), normalize_checkbox), cascade_response_plot,
)

# Final layout
dashboard = pn.Row(left_column, pn.Spacer(width=20), widgets)

To view the dashboard, we simple enter the name of the Panel object of the dashboard in a code cell and execute it. If we attach a `.servable()` call to it, it will still render in the notebook, but the graphic is also servable from the command line.

In [13]:
dashboard.servable()

Row
    [0] Column
        [0] Row
            [0] Spacer(width=30)
            [1] Checkbox(name='Normalize')
        [1] ParamFunction(function)
    [1] Spacer(width=20)
    [2] Column(width=150)
        [0] Spacer(height=10)
        [1] FloatSlider(end=2, name='log₁₀ β', start=-1, value_throttled=0)
        [2] FloatSlider(end=2, name='log₁₀ γ', start=-1, value_throttled=0)
        [3] FloatSlider(end=10, name='nx', start=0.1, value=2, value_throttled=2)
        [4] FloatSlider(end=10, name='ny', start=0.1, value=2, value_throttled=2)
        [5] FloatSlider(end=20, name='t_max', start=1, value=10, value_throttled=10)
        [6] FloatSlider(end=2, name='log₁₀ x0', start=-1, value=0.3010299956639812, value_throttled=0.3010299956639812)

To serve the dashboard from the command line, you can enter the following:

    panel serve name_of_the_notebook.ipynb --show
    
This will open your default browser (Firefox or Chrome recommended) and the dashboard will be displayed along in a browser tab and be fully interactive.

Importantly, when exploring the plot interactively, we see that when Y cooperatively activates Z (that is, $n_y$ is large), the delay is longer. The delay is also naturally longer as $\gamma$ gets small. This makes sense, since $\gamma$ is the ratio of the decay rate of Z to that of Y. As we saw in the previous lesson, the decay rate sets the speed of the response, and if Z decays more slowly than Y, its dynamics will be slower.

Note that the above conclusions are clearer when we plot the normalized curves.

## Speeding up interactions and Hill functions

The process of building the interactive plot may have seemed like a lot of steps. However, if we look back at what we did to build it, we did the following.

1. Wrote a function to make a plot.
2. Built widgets to control the parameters of the plot.
3. Make a layout positioning the plot(s) and widgets.

You pretty much do step 1 whenever you make any plot. Panel makes the interactivity easy by providing widgets and function decorators.

When we built the plots above, we wrote a function that constructs a new plot every time we change a parameter value. This works well, but it can become slow, since we are re-generating the entire graphic with each update. To have faster responses, we could instead update only the *data* that is plotted.

To illustrate how this works, and to get a feel for Hill functions in the meantime, we can look at how tuning the Hill coefficient and Hill $k$ values affect the shape of a Hill function.

We start by writing a function to make a plot of an increasing (in blue) and decreasing (in orange) Hill function. We assume that $k$ and the concentration $x$ are in the same (unspecified) units. We will not need any Panel decorator here, as will become clear momentarily.

In [14]:
def hill_plot(k=1, n=1):
    # Set up plot
    p = bokeh.plotting.figure(
        frame_width=450,
        frame_height=250,
        x_axis_label="x",
        y_axis_label="f(x;k,n)",
        x_range=[0, 10],
    )

    # Set up empty data source
    x = np.linspace(0, 10, 400)
    y_decreasing = 1 / (1 + (x / k) ** n)
    y_increasing = 1 - y_decreasing
    source = bokeh.models.ColumnDataSource(
        dict(x=x, y_increasing=y_increasing, y_decreasing=y_decreasing)
    )

    # Populate glyphs
    p.line("x", "y_increasing", source=source, line_width=2, color=colors[0])
    p.line("x", "y_decreasing", source=source, line_width=2, color=colors[1])

    return p

Next, we can build sliders for $k$ and $n$.

In [15]:
k_slider = pn.widgets.FloatSlider(
    name="k", start=0.1, end=10, step=0.1, value=1, width=150
)
n_slider = pn.widgets.FloatSlider(
    name="n", start=0.1, end=10, step=0.1, value=1, width=150
)

Next, we need to make sure the Bokeh plot is in a Panel **pane**, so that we can link it to the slider, which we need to do explicitly since we're updating the data source and not just replotting.

In [16]:
p_pane = pn.pane.Bokeh(hill_plot())

Next, we need a **callback**. This is a function that gets executed every time the slider changes. In our previous example, by default, the callback was to regenerate the entire plot. Here, we update the data source in our callback. We have to get into the guts of the Bokeh figure, pulling out the glyph renderer and adjusting its properties, including its data source.

In [17]:
def k_callback(target, event):
    # New value of k
    k = event.new

    # Fetch value of n
    n = n_slider.value

    # Extract data source
    gr = target.object.renderers[0]
    source = gr.data_source

    # Update data source
    x = source.data["x"]
    source.data["y_decreasing"] = 1 / (1 + (x / k) ** n)
    source.data["y_increasing"] = 1 - source.data["y_decreasing"]


def n_callback(target, event):
    # Fetch value of k
    k = k_slider.value

    # New value of n
    n = event.new

    # Extract data source
    gr = target.object.renderers[0]
    source = gr.data_source

    # Update data source
    x = source.data["x"]
    source.data["y_decreasing"] = 1 / (1 + (x / k) ** n)
    source.data["y_increasing"] = 1 - source.data["y_decreasing"]

Finally, we need to **link** the sliders with the plot.

In [18]:
k_slider.link(p_pane, callbacks={'value': k_callback})
n_slider.link(p_pane, callbacks={'value': n_callback})

Watcher(inst=FloatSlider(end=10, name='n', start=0.1, value=1, value_throttled=1, width=150), cls=<class 'panel.widgets.slider.FloatSlider'>, fn=<function Reactive.link.<locals>.link at 0x7fb6314561f0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

The result that was printed to the screen simply says that we have set up a **watcher** so that the plot will get updated whenever the slider is changed. Now, we can look at our result! (Note that we added `.servable()` so we can serve up this graphic as well. If we serve this notebook from the command line, both this interactive graphic and the dashboard for the cascade we built above will be displayed.)

In [19]:
pn.Column(
    pn.Spacer(height=30),
    pn.Row(
        p_pane,
        pn.Spacer(width=15),
        pn.Column(pn.Spacer(height=40), k_slider, pn.Spacer(height=15), n_slider),
    ),
).servable()

Column
    [0] Spacer(height=30)
    [1] Row
        [0] Bokeh(Figure)
        [1] Spacer(width=15)
        [2] Column
            [0] Spacer(height=40)
            [1] FloatSlider(end=10, name='k', start=0.1, value=1, value_throttled=1, width=150)
            [2] Spacer(height=15)
            [3] FloatSlider(end=10, name='n', start=0.1, value=1, value_throttled=1, width=150)

The response of sliders constructed this way can be much quicker than in the simpler method we did before, which regenerates the plot each time the slider changes value. However, as you can see from this example, you have to write more code to implement the faster-responding sliders. Going forward, we will almost always use the simpler construction, where we simple write a function that generates a plot and include that in our Panel layout, though in some applications, speed matters.

## Duration of input

Coming back to our cascade circuit, now imagine that the input is a pulse of duration $\tau$.  We can write a function for this and plot it.

In [20]:
def x_pulse(t, t_0, tau, x_0):
    """
    Returns x value for a pulse beginning at t = 0 
    and ending at t = t_0 + tau.
    """
    return np.logical_and(t >= t_0, t <= (t_0 + tau)) * x_0


# Plot the pulse
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless x",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, x_pulse(t, 1.0, 2.0, 2), line_width=2, color=colors[2])

# Show plot
bokeh.io.show(p)

If we want to solve the ODEs for a pulsed input, we need to have a way to pass this function as a parameter. Fortunately, we can pass functions as arguments in Python! So, we write a new function that takes `x_fun`, the function describing $x(t)$ as an argument, as well as `x_args`, the set of parameters passed into `x_fun`.

In [21]:
def cascade_rhs_x_fun(yz, t, beta, gamma, n_x, n_y, x_fun, x_args):
    """
    Right hand side for cascade X -> Y -> Z.  Return dy/dt and dz/dt.
    
    x_fun is a function of the form x_fun(t, *x_args), so x_args is a tuple
    containing the arguments to pass to x_fun.
    """
    # Compute x
    x = x_fun(t, *x_args)

    # Return cascade RHS with this value of x
    return cascade_rhs(yz, t, beta, gamma, n_x, n_y, x)

With this in hand, we can now solve for a pulse.  We will have a pulse during $1 \le t \le 5$.

In [22]:
# Set up parameters for the pulse (on at t = 1, off at t = 5, x_0 = 2)
x_args = (1.0, 4.0, 2.0)

# Package parameters into a tuple
args = (beta, gamma, n_x, n_y, x_pulse, x_args)

# Integrate ODEs
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, y, line_width=2, color=colors[0], legend_label="y")
p.line(t, z, line_width=2, color=colors[1], legend_label="z")

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

Let's see what happens when we do a shorter pulse, this time with $1 \le t \le 1.7$.

In [23]:
# Set up parameters for the pulse (on at t = 1, off at t = 1.7, x_0 = 2)
x_args = (1.0, 0.7, 2.0)

# Package parameters into a tuple
args = (beta, gamma, n_x, n_y, x_pulse, x_args)

# Integrate ODEs
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, y, line_width=2, color=colors[0], legend_label="y")
p.line(t, z, line_width=2, color=colors[1], legend_label="z")

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

We see that Z basically does not respond strongly to a short pulse.  The delay of the circuit allows short pulses to be ignored, but large pulses to be detected and responded to.

## Really short pulses and a lesson about scipy.integrate.odeint()

Now, we will take a brief interlude to learn an important lesson about the algorithm of `scipy.integrate.odeint()` and its use in these applications. We will consider a very brief pulse, $1 \le t \le 1.05$.

In [24]:
# Set up parameters for the pulse (on at t = 1, off at t = 1.05, x_0 = 2)
x_args = (1.0, 0.05, 2.0)

# Package parameters into a tuple
args = (beta, gamma, n_x, n_y, x_pulse, x_args)

# Integrate ODEs
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, y, line_width=2, color=colors[0], legend_label="y")
p.line(t, z, line_width=2, color=colors[1], legend_label="z")

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

Uh oh!  Something went wrong, since the Y signal never went up. This exposes an important issue with the algorithm used by `scipy.integrate.odeint()`. The Hindmarsh-Petzold algorithm uses variable step sizes so that it takes long steps when the system is not changing much and short steps when it is. Therefore, if we have a long period of no changes (leading up to $t = 1$), the step sizes taken by the solver will increase, and we'll step right over the pulse.

So, it is in general good practice to explicitly take into account discontinuities in the parameters over time. In this case, we would use `scipy.integrate.odeint()` to integrate to the pulse and use the end point of that as the initial condition of a new solution during the pulse. Then, at the end of the pulse, we start again.  Let's try again using this method.

In [25]:
# Integrate prior to pulse
t_before_pulse = np.linspace(0, 1.0, 20)
args = (beta, gamma, n_x, n_y, 0.0)
yz_0 = np.array([0.0, 0.0])
yz_before_pulse = scipy.integrate.odeint(
    cascade_rhs, yz_0, t_before_pulse, args=args
)

# Integrate during pulse
t_during_pulse = np.linspace(1.0, 1.05, 50)
args = (beta, gamma, n_x, n_y, 2.0)
yz_0 = yz_before_pulse[-1]
yz_during_pulse = scipy.integrate.odeint(
    cascade_rhs, yz_0, t_during_pulse, args=args
)

# Integrate after pulse
t_after_pulse = np.linspace(1.05, 5, 50)
args = (beta, gamma, n_x, n_y, 0.0)
yz_0 = yz_during_pulse[-1]
yz_after_pulse = scipy.integrate.odeint(
    cascade_rhs, yz_0, t_after_pulse, args=args
)

# Piece together solution
t = np.concatenate((t_before_pulse, t_during_pulse[1:], t_after_pulse[1:]))
yz = np.vstack(
    (yz_before_pulse, yz_during_pulse[1:, :], yz_after_pulse[1:, :])
)
y, z = yz.transpose()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 5],
)

# Populate glyphs
p.line(t, y, line_width=2, color=colors[0], legend_label="y")
p.line(t, z, line_width=2, color=colors[1], legend_label="z")

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

Much better. Dealing with discontinuities like this while solving systems of ODEs are facilitated by **event handling**. Unfortunately, `scipy.integrate.odeint()` does not allow for event handling, and we need to take the less elegant approach we just showed.

Note that `scipy.integrate.solve_ivp()` *does* have event handling capabilities. However, we will seldom need event handing for our purposes, and it is still advantages to use `scipy.integrate.odeint()` due to its low overhead.

## Periodic input

Finally, we will consider the case where we have periodic forcing of the circuit. We will do this for highly cooperative activation by Y, taking $n_y = 10$. Recall that this gives a longer time delay.

We first write a function for the forcing, `x_fun`, which is periodic with frequency $f$.

In [26]:
def x_periodic(t, f, x_0):
    """
    Returns x value for periodic forcing of amplitude x_0 and frequency f.
    """
    if type(f) in [float, int]:
        return x_0 * (1 + np.sin(f * t))
    else:
        sin_sum = np.zeros_like(t)
        for freq, amp in zip(f, x_0):
            sin_sum += amp * (1 + np.sin(freq * t))
        return sin_sum


# Plot the forcing
t = np.linspace(0, 10, 500)

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, x_periodic(t, 5, 2.0), line_width=2, color=colors[2])

# Show plot
bokeh.io.show(p)

Let's see how the circuit responds to a low-frequency input.

In [27]:
# Set up parameters for periodic forcing with f = 0.5 and x_0 = 2.
x_args = (0.5, 2.0)

# Package parameters into a tuple, now with high ultrasensitivity
n_y = 10
args = (beta, gamma, n_x, n_y, x_periodic, x_args)

# Time points
t = np.linspace(0, 40, 300)

# Initial condition
yz_0 = np.array([0.0, 0.0])

# Integrate ODES
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# x
x = x_periodic(t, *x_args)
x /= x.max()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, y, line_width=2, color=colors[0], legend_label="y")
p.line(t, z, line_width=2, color=colors[1], legend_label="z")
p.line(
    t,
    x,
    line_width=2,
    color=colors[2],
    alpha=0.2,
    legend_label="x (normalized)",
    line_join="bevel",
)

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

We roughly follow the forcing with some lag.  Now, for high-frequency forcing, we have a different response.

In [28]:
# Set up parameters for periodic forcing with f = 5 and x_0 = 2.
x_args = (5.0, 2.0)

# Package parameters into a tuple
args = (beta, gamma, n_x, 10, x_periodic, x_args)

# Time points
t = np.linspace(0, 25, 600)

# Initial condition
yz_0 = np.array([0.0, 0.0])

# Integrate ODES
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# x
x = x_periodic(t, *x_args)
x /= x.max()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
    x_range=[0, 10],
)

# Populate glyphs
p.line(
    t, y, line_width=2, color=colors[0], legend_label="y", line_join="bevel"
)
p.line(
    t, z, line_width=2, color=colors[1], legend_label="z", line_join="bevel"
)
p.line(
    t,
    x,
    line_width=2,
    color=colors[2],
    alpha=0.2,
    legend_label="x (normalized)",
    line_join="bevel",
)

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

We see that Z does not really respond to high frequency forcing, even though the forcing is with the same amplitude. This gives us a design principle, that **a cascade can filter out high frequency fluctuations**. We can see this by adding another frequency to the signal.

In [29]:
# Set up parameters for periodic forcing with f = 5 and x_0 = 2.
x_args = ((0.5, 10.0), (0.5, 2.0))

# Package parameters into a tuple
args = (beta, gamma, n_x, 10, x_periodic, x_args)

# Time points
t = np.linspace(0, 25, 600)

# Initial condition
yz_0 = np.array([0.0, 0.0])

# Integrate ODES
yz = scipy.integrate.odeint(cascade_rhs_x_fun, yz_0, t, args=args)

# Pluck out y and z
y, z = yz.transpose()

# x
x = x_periodic(t, *x_args)
x /= x.max()

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="dimensionless time",
    y_axis_label="dimensionless y, z",
)

# Populate glyphs
p.line(
    t, y, line_width=2, color=colors[0], legend_label="y", line_join="bevel"
)
p.line(
    t, z, line_width=2, color=colors[1], legend_label="z", line_join="bevel"
)
p.line(
    t,
    x,
    line_width=2,
    color=colors[2],
    alpha=0.2,
    legend_label="x (normalized)",
    line_join="bevel",
)

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

Even though the high frequency part of the forcing has a bigger amplitude, the signal in *z* responds predominantly to the low frequency part of the signal.

## A challenge

In the above analysis of periodic forcing, I made a fresh plot each time of the response to an oscillating input so that the plots would display nicely in the HTML rendered version of this notebook. Can you make the sample plot(s) by making a single interactive plot?

## Computing environment

In [30]:
%load_ext watermark
%watermark -v -p numpy,scipy,bokeh,panel,colorcet,jupyterlab

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

numpy     : 1.19.2
scipy     : 1.6.2
bokeh     : 2.3.0
panel     : 0.10.3
colorcet  : 2.0.6
jupyterlab: 3.0.11

